In [ ]:
from W_utility.log import * 
from W_utility.file import * 

from NLP.porter2 import *
from NLP.word import *
from NLP.sentence import *
from NLP.sentence_keywords import * 

In [ ]:
import re, math, csv
import W_utility.file as ufile
from NLP import sentence
from NLP import sentence_keywords

In [ ]:
# Declare vars to empty
greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation = "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", "", ""

def init_features ():
    # Import csv data as dictionary 
    feature_set = ufile.read_csv_as_dict ('data\\numeric_features.csv', 0, 1, True)
    
    # Declare Global variables from the imported data
    global greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation
    greater, greater_equal, greater_equal2, lower, lower_equal, lower_equal2, equal, between, selects, connect, features, temporal, temporal_con, error1, error2, symbols, numbers, unit_special, unit_ori, unit_ori_s, unit_exp, negation = \
    feature_set["greater"], feature_set["greater_equal"], feature_set["greater_equal2"], feature_set["lower"], feature_set["lower_equal"], feature_set["lower_equal2"], feature_set["equal"], feature_set["between"], feature_set["selects"], feature_set["connect"], feature_set["features"], feature_set["temporal"], feature_set["temporal_con"], feature_set["error1"], feature_set["error2"], feature_set["symbols"], feature_set["numbers"], feature_set["unit_special"], feature_set["unit_ori"], feature_set["unit_ori_s"], feature_set["unit_exp"], feature_set["negation"]
    
    # ADD plural form of words in temporal 
    temporal = temporal + '|' + temporal.replace('|', 's|') + 's'
    
    # Create a new varibale that consists of words in unit_ori, unit_ori_s, plural form of words in unit_ori_s, temporal
    unit = (unit_ori + "|" + unit_ori_s.replace("|", "s|") + "s|" + unit_ori_s + "|" + temporal)
    return ""

In [ ]:
init_features()

In [ ]:
# Text replace sor sub
def preprocessing (text):
    # handle special characters
    text = text.encode().decode('ascii', 'ignore')
    
    # Strip() is used to remove whitespace here. #Strip is used to remove leading & trailing characters. If omitted or None, the chars argument defaults to removing whitespace.
    # '#' is used the 'example data diabetes_Type 1.csv' to end a sentence. Hence \n\n is replaced with #
    text = text.strip().replace('\n\n', '#') 
    
    # \n is replced with empty
    text = text.replace ('\n', '')
    
    text = text.replace(u'＝','=').replace(u'＞', '>').replace(u'＜','<').replace(u'≤','<=').replace (u'≥','>=').replace(u'≦','<=').replace(u'≧','>=').replace(u'mm³','mm^3').replace(u'µl','ul').replace(u'µL','ul').replace(u'·','').replace(u'‐','-').replace(u'—','-')
    
    # Double brackets are replced with single brackets
    text = text.replace('((', '(').replace('))', ')')
    
    
    # "105&200" converted to "105 - 200"
    # "105 & 200" converted to "105 - 200"
    # & or / or ~ is replced with -
    # \1 corresponds to 1st char/string i.e (\d+) 
    # \5 corresponds to 5th char/string i.e (\d+)
    # ( |) - corresponds to if space is present or not
    text = re.sub('(\d+)( |)(~|/|&|\|)( |)(\d+)',r'\1 - \5',text) # e.g., '10~20' to '10 ~ 20'
    
    # Commas in only thousands is replaced (Makes sense as if r"(\d+),(\d+)" is used instead, July24,2017 will be changed to July242017 & in 10/19,10/20 will be changed to 10/1910/20)
    text = re.sub(r"(\d+),(\d{3})", r'\1\2', text) # 10,123 to 10123
    
    # Should be run only using DateTime function
    # If only 1 or 2 digits are present after 'digits followed by comma' 
    # Not needed to run this 
    text = re.sub(r"(\d+),(\d{1,2})", r'\1.\2', text) # 10,1 to 10.1
    
    # r"between (\d+), (\d{1,2}) (and|or) ", r'between \1.\2 \3 ' MODIFIED TO  r"between (\d+),( |)(\d{1,2}) (and|or) ", r'between \1.\3 \4 '
    # Not needed to run this
    text = re.sub(r"between (\d+),( |)(\d{1,2}) (and|or) ", r'between \1.\3 \4 ', text) # 'between 7, 5 and ' to 'between 7.5 and '
    
    # Within a text if there double spaces - replace it with a single space
    while '  ' in text:
        text = text.replace('  ',' ')
    
    # Global variable - Declared in init_features (comma before but|and|or is removed )
    # avoid connected values separated by splitting, e.g., ", but below 10%"
    text = re.sub(", ("+connect+") ", r' \1 ', text) # 

    # text is converted to lowercase
    return text.lower()

In [ ]:
# XXX #
### example data diabetes_Type 1 contains some key words exclusion, inclusion, keywords mentioned in in_fea, ex_fea
# This can be ignored for our data (As we are not splitting our data to Inclusion, Exclusion subgroups)
def split_text_inclusion_exclusion(otext):
    in_fea = 'inclusion criteria:|key inclusion criteria|inclusion criteria [^:#;\.]+:|inclusion:|(?<!(\w| ))inclusion criteria\W\W|inclusion for'
    ex_fea = 'exclusion criteria:|key exclusion criteria|exclusion criteria [^:#;\.]+:|exclusion:|(?<!(\w| ))exclusion criteria\W\W|exclusion for'
   
    # Declare vars to empty, to use them in for loops
    in_text, ex_text = '', ''
    
    # in_bool is initially set to true but modified in While loop
    in_bool = True
    
    # text is converted to lowercase
    text = otext.lower()
    
    # If text is NOT empty
    while text != '':
        # if in_bool is True:
        if in_bool:
            n_pos = re.search('('+ex_fea+')',text)
            if n_pos is not None:
                in_text += text[0:n_pos.start()]
                text = text[n_pos.start():]
            else:
                in_text += text[0:]
                text = ''
        
        # if in_bool is False
        else:
            n_pos = re.search('('+in_fea+')',text)
            if n_pos is not None:
                ex_text += text[0:n_pos.start()]
                text = text[n_pos.start():]
            else:
                ex_text += text[0:]
                text = ''
        in_bool = False if in_bool else True
    
    sections_text =[]
    if in_text !='': sections_text.append(["Inclusion", in_text])
    if ex_text !='': sections_text.append(["Exclusion", ex_text])    
    return sections_text

In [ ]:
# XXX #
#====find expression candidates according to pre-defined feature list
# This whole function can be replaced with the below one
def extract_candidates_numeric (text):
    # process text
    sections_text = split_text_inclusion_exclusion(text)
    
    sections_num = []
    candidates_num = []
      
    # Each sections i.e ['inclusion','its text']
    for section_text in sections_text:        
        # Within each paragraph (section_text[1]) , split sentences based on characters - [#!?.;]\s (/s is whitespace character)
        sentences = sentence.sentence_splitting_symbols(section_text[1], "[#!?.;]\s", 1)
        for sent in sentences:
            #Strip is used to remove leading & trailing characters. Here '- ' is removed 
            sent = sent.strip().strip('- ')
            if sent == '':
                continue
                
            digit = re.search("(?<!(\w))\d+", sent)
            if digit:
                sections_num.append(section_text[0])
                candidates_num.append(sent)

    return (sections_num, candidates_num)

In [ ]:
### Find sentences that have numeric values in them. (splits paragraph into sentences and extracts only those with numerics in them)
def extract_candidates_numeric (text):
    
    # Split text into sentences
    sentences = sentence.sentence_splitting_symbols(text, "[#!?.;]\s", 1)
    # Create a empty array
    candidates_num = []
    
    for sent in sentences:
        #Strip is used to remove leading & trailing characters. Here '- ' is removed 
        sent = sent.strip().strip('- ')
        
        if sent == '':
            continue

        digit = re.search("(?<!(\w))\d+", sent) # Searching for a digit
        if digit:
            candidates_num.append(sent) # Append sentences that have numeric values, to the array
            
    return (candidates_num)

In [ ]:
# XXX #
# Extract only sentences with a particular name/string/word. If name_list is empty then there is NO restriction on which sentences to extract
# Function modified to meet extract_candidates_numeric new function
name_list = ""
def extract_candidates_name (candidates_num, name_list):
    
    candidates = []
    
    # If name_list="heart rate | temperature | blood pressure"; name_list.split('|') splits to give ['heart rate ', ' temperature ', ' blood pressure']
    names = name_list.split('|') 
    for i in range(len(candidates_num)):            
        for name in names:
           if name in candidates_num[i]: #If string/word is in the sentence it is appended
                candidates.append(candidates_num[i]) # Append 
                break
            
            # else statement is NOT present as if name_list is empty all sentences are extracted

    return (candidates)

In [ ]:
#====identify expressions and formalize them into labels "<VML(tag) L(logic, e.g., greater_equal)=X U(unit)=X>value</VML>"
def formalize_expressions (candidate):
    text = candidate
    csvfile = open('data\\rules.csv', 'r')
    reader = csv.reader(csvfile)
    now_pattern = "preprocessing"

    for i,pattern in enumerate(reader):
        source_pattern = pattern[0]
        target_pattern = pattern[1]
        pattern_function = pattern[2]

        if(pattern_function == "process_numerical_values" and pattern_function != now_pattern):
            matchs = re.findall('<Unit>([^<>]+)</Unit>', text)
            for match in matchs: text = text.replace(match, match.replace(' / ', '/').replace(' - ','-'))

        if(pattern_function == "process_special_logics" and pattern_function != now_pattern):
            # process 'select' expression, use the first one
            selects = "X \( X \)|X \( equal to X \)|X \( = X\)"
            aselect = selects.split('|')
            for selec in aselect:
                selec = selec.replace('X', '<VML Unit([^<>]+)>([^<>]+)</VML>')
                text = re.sub(selec, r'<VML Unit\1>\2</VML>', text) #

            #  process 'between' expressions
            between = "range of X to X|range X to X|range X - X|between X to X|between X and X|between Xand X|between X - X|between X & X|from X to X|within X to X|start X and X|X through X|of X and X|>= X and X|> X and X|of X to X|>= X to <= X|>= X to X|> X to X|X - <= X|X to X|X - X"
            betweens = between.split('|')
            for betw in betweens:
                betw = betw.replace('X', '<VML Unit([^<>]+)>([^<>]+)</VML>')
                text = re.sub(betw, r'<VML Logic=greater_equal Unit\1>\2</VML> - <VML Logic=lower_equal Unit\3>\4</VML>', text) #
        text = re.sub(source_pattern, target_pattern, text)
        now_pattern = pattern_function

    csvfile.close()
    return text

In [ ]:
import pandas as pd
ExampleData = pd.read_csv("data/example data diabetes_Type 1.csv",header=None)

In [ ]:
pd.set_option('display.max_colwidth',-1)
ExampleData